## Глава 0: Краткое руководство

### Предупреждение о спойлерах

В этой короткой главе мы сразу перейдем к делу и выполним тонкую настройку небольшой языковой модели, а именно Microsoft Phi-3 Mini 4K Instruct, для перевода английского языка на манер речи Йоды. Эту начальную главу можно рассматривать как рецепт, которому можно просто следовать. Это глава в духе «сначала стреляй, потом задавай вопросы».

Вы научитесь:
*   загружать квантизированную модель с использованием `BitsAndBytes`;
*   настраивать низкоранговые адаптеры (LoRA) с помощью `peft` от Hugging Face;
*   загружать и форматировать набор данных;
*   выполнять тонкую настройку модели с помощью `SFTTrainer` (supervised fine-tuning trainer) из библиотеки `trl` от Hugging Face;
*   использовать дообученную модель для генерации нескольких предложений.

### Настройка окружения

Для обеспечения лучшей воспроизводимости во время обучения используйте зафиксированные версии, указанные ниже (те же, что использовались в книге):

In [1]:
# Оригинальные версии
#!pip install transformers==4.46.2 peft==0.13.2 accelerate==1.1.1 trl==0.12.1 bitsandbytes==0.45.2 datasets==3.1.0 huggingface-hub==0.26.2 safetensors==0.4.5 pandas==2.2.2 matplotlib==3.8.0 numpy==1.26.4
# Обновленные версии - Октябрь 2025
# !pip install transformers==4.56.1 peft==0.17.0 accelerate==1.10.0 trl==0.23.1 bitsandbytes==0.47.0 datasets==4.0.0 huggingface-hub==0.34.4 safetensors==0.6.2 pandas==2.2.2 matplotlib==3.10.0 numpy==2.0.2


In [2]:
# Если вы используете Colab
#!pip install datasets bitsandbytes trl


In [3]:
# Если вы используете Jupyter Template от runpod.io
#!pip install datasets bitsandbytes trl transformers peft huggingface-hub accelerate safetensors pandas matplotlib


## Импорт библиотек

In [4]:
import os
import torch
from contextlib import nullcontext
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer


## Загрузка квантизированной базовой модели

Мы начнем с загрузки квантизированной модели, чтобы она занимала меньше места в оперативной памяти GPU. Квантизированная модель заменяет исходные веса приближенными значениями, представленными меньшим количеством бит. Самый простой и прямой способ квантизации модели — преобразовать ее веса из 32-битных чисел с плавающей запятой (FP32) в 4-битные числа с плавающей запятой (NF4). Это простое, но мощное изменение уже **сокращает объем памяти, занимаемый моделью**, примерно в восемь раз.

Мы можем использовать экземпляр `BitsAndBytesConfig` в качестве аргумента `quantization_config` при загрузке модели с помощью метода `from_pretrained()`. Чтобы сохранить гибкость и позволить вам опробовать любую другую модель по вашему выбору, мы используем `AutoModelForCausalLM` от Hugging Face. Репозиторий, который вы выберете, определяет загружаемую модель.

Без дальнейших церемоний, вот наша квантизированная модель, которая загружается:

In [5]:
bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float32
)
repo_id = 'microsoft/Phi-3-mini-4k-instruct'
model = AutoModelForCausalLM.from_pretrained(repo_id,
                                             device_map="cuda:0",
                                             quantization_config=bnb_config
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<blockquote class="note">
  <p>
    <em>"Phi-3-Mini-4K-Instruct — это легкая, современная открытая модель с 3.8 млрд параметров, обученная на наборах данных Phi-3, которые включают как синтетические данные, так и отфильтрованные общедоступные данные веб-сайтов с акцентом на высокое качество и свойства, насыщенные логическими рассуждениями. Модель принадлежит к семейству Phi-3, мини-версия представлена в двух вариантах: 4K и 128K, что указывает на поддерживаемую длину контекста (в токенах)."</em>
    <br>
    Источник: <a href="https://huggingface.co/microsoft/Phi-3-mini-4k-instruct">Hugging Face Hub</a>
  </p>
</blockquote>

In [6]:
print(model.get_memory_footprint()/1e6)


2206.341312


Даже будучи квантизированной, модель по-прежнему занимает чуть более 2 гигабайт оперативной памяти. Процедура **квантизации** применяется к **линейным слоям внутри блоков Transformer-декодера** (в некоторых случаях также называемых "слоями"):

In [7]:
model


Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_

**Квантизированную модель** можно использовать напрямую для вывода (инференса), но **невозможно продолжить ее обучение**. Эти назойливые слои `Linear4bit` занимают гораздо меньше места, что и является целью квантизации; однако мы не можем их обновлять.

Нам нужно добавить кое-что еще в нашу смесь — немного адаптеров.

## Настройка низкоранговых адаптеров (LoRA)

Низкоранговые адаптеры могут быть подключены к каждому из квантизированных слоев. **Адаптеры** по большей части представляют собой **обычные слои `Linear`**, которые можно легко обновлять обычным образом. Хитрый трюк здесь заключается в том, что эти адаптеры **значительно меньше** по размеру, чем квантизированные слои.

Поскольку **квантизированные слои заморожены** (их нельзя обновлять), настройка **адаптеров LoRA** на квантизированной модели резко **сокращает общее количество обучаемых параметров** до всего 1% (или менее) от исходного размера.

Мы можем настроить адаптеры LoRA в три простых шага:

*   Вызвать `prepare_model_for_kbit_training()` для *повышения численной стабильности* во время обучения.
*   Создать экземпляр `LoraConfig`.
*   Применить конфигурацию к квантизированной базовой модели с помощью метода `get_peft_model()`.

Давайте попробуем сделать это с нашей моделью:

In [8]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,                   # ранг адаптера; чем меньше, тем меньше параметров нужно обучать
    lora_alpha=16,         # множитель, обычно 2*r
    bias="none",           # ВНИМАНИЕ: обучение смещений *изменяет* поведение базовой модели
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    # Более новые модели, такие как Phi-3 на момент написания, могут требовать
    # ручной установки целевых модулей
    target_modules=['o_proj', 'qkv_proj', 'gate_up_proj', 'down_proj'],
)

model = get_peft_model(model, config)
model


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3Attention(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (

Вывод для остальных трех слоев LoRA (`qkv_proj`, `gate_up_proj` и `down_proj`) был скрыт для сокращения вывода.

<blockquote class="warning">
  <p>
    Возникла ли у вас следующая ошибка?
    <br>
    <br>
    <tt>ValueError: Please specify `target_modules` in `peft_config`</tt>
    <br>
    <br>
    Скорее всего, вам не нужно указывать <tt>target_modules</tt>, если вы используете одну из хорошо известных моделей. Библиотека <tt>peft</tt> позаботится об этом, *автоматически выбирая подходящие цели*. Однако может возникнуть разрыв между временем выпуска популярной модели и обновлением библиотеки. Поэтому, если вы получили указанную выше ошибку, найдите квантизированные слои в вашей модели и перечислите их имена в аргументе <tt>target_modules</tt>.
  </p>
</blockquote>

Квантизированные слои (`Linear4bit`) превратились в модули `lora.Linear4bit`, где сам квантизированный слой стал `base_layer` с добавленными к нему обычными слоями `Linear` (`lora_A` и `lora_B`).

Эти дополнительные слои должны были бы лишь незначительно увеличить модель. Однако **функция подготовки модели** (`prepare_model_for_kbit_training()`) преобразовала **все неквантизированные слои в полную точность (FP32)**, что привело к увеличению размера модели на 30%:

In [9]:
print(model.get_memory_footprint()/1e6)


2651.074752


Поскольку большинство параметров заморожено, лишь крошечная доля от общего количества параметров в настоящее время является обучаемой, благодаря LoRA!

In [10]:
trainable_parms, tot_parms = model.get_nb_trainable_parameters()
print(f'Обучаемые параметры:             {trainable_parms/1e6:.2f}M')
print(f'Всего параметров:                {tot_parms/1e6:.2f}M')
print(f'Доля обучаемых параметров:       {100*trainable_parms/tot_parms:.2f}%')


Обучаемые параметры:             12.58M
Всего параметров:                3833.66M
Доля обучаемых параметров:       0.33%


Модель готова к тонкой настройке, но нам все еще не хватает одного ключевого компонента: нашего набора данных.

## Форматирование вашего набора данных

<blockquote style="quotes: none !important;">
  <p>
    <em>"Как Йода, говорить, ты должен. Хррммм."</em>
    <br>
    <br>
    Мастер Йода
  </p>
</blockquote>

Набор данных [`yoda_sentences`](https://huggingface.co/datasets/dvgodoy/yoda_sentences) состоит из 720 предложений, переведенных с английского на манер речи Йоды. Набор данных размещен на Hugging Face Hub, и мы можем легко загрузить его с помощью метода `load_dataset()` из библиотеки Hugging Face `datasets`:

In [11]:
dataset = load_dataset("dvgodoy/yoda_sentences", split="train")
dataset


Dataset({
    features: ['sentence', 'translation', 'translation_extra'],
    num_rows: 720
})

Набор данных имеет три столбца:

*   исходное предложение на английском (`sentence`);
*   базовый перевод на манер речи Йоды (`translation`);
*   расширенный перевод, включающий типичные междометия `Yesss` и `Hrrmm` (`translation_extra`).

In [12]:
dataset[0]


{'sentence': 'The birch canoe slid on the smooth planks.',
 'translation': 'On the smooth planks, the birch canoe slid.',
 'translation_extra': 'On the smooth planks, the birch canoe slid. Yes, hrrrm.'}

`SFTTrainer`, который мы будем использовать для тонкой настройки модели, может автоматически обрабатывать наборы данных в **диалоговом** формате.

```
{"messages":[
  {"role": "system", "content": "<общие директивы>"},
  {"role": "user", "content": "<текст промпта>"},
  {"role": "assistant", "content": "<идеально сгенерированный текст>"}
]}
```

***

**ВАЖНОЕ ОБНОВЛЕНИЕ**: К сожалению, в более новых версиях библиотеки `trl` формат "instruction" больше не поддерживается должным образом, что приводит к тому, что шаблон чата не применяется к набору данных. Чтобы избежать этой проблемы, мы можем преобразовать набор данных в "диалоговый" формат.

***

In [13]:
# Адаптировано из trl.extras.dataset_formatting.instructions_formatting_function
# Преобразует набор данных из формата промпт/завершение (больше не поддерживается)
# в диалоговый формат
def format_dataset(examples):
    if isinstance(examples["prompt"], list):
        output_texts = []
        for i in range(len(examples["prompt"])):
            converted_sample = [
                {"role": "user", "content": examples["prompt"][i]},
                {"role": "assistant", "content": examples["completion"][i]},
            ]
            output_texts.append(converted_sample)
        return {'messages': output_texts}
    else:
        converted_sample = [
            {"role": "user", "content": examples["prompt"]},
            {"role": "assistant", "content": examples["completion"]},
        ]
        return {'messages': converted_sample}


In [14]:
dataset = dataset.rename_column("sentence", "prompt")
dataset = dataset.rename_column("translation_extra", "completion")
dataset = dataset.map(format_dataset)
dataset = dataset.remove_columns(['prompt', 'completion', 'translation'])
messages = dataset[0]['messages']
messages


[{'content': 'The birch canoe slid on the smooth planks.', 'role': 'user'},
 {'content': 'On the smooth planks, the birch canoe slid. Yes, hrrrm.',
  'role': 'assistant'}]

### Токенизатор

Прежде чем перейти к непосредственному обучению, нам все еще необходимо **загрузить токенизатор, соответствующий нашей модели**. Токенизатор является важной частью этого процесса, определяя, как преобразовывать текст в токены тем же способом, который использовался при обучении модели.

Для инструктивных/чат-моделей токенизатор также содержит соответствующий **шаблон чата**, который определяет:

*   Какие **специальные токены** следует использовать и где их размещать.
*   Где должны размещаться системные директивы, промпт пользователя и ответ модели.
*   Что такое **промпт генерации**, то есть специальный токен, который запускает ответ модели (подробнее об этом в разделе "Запрос к модели").

***

**ВАЖНОЕ ОБНОВЛЕНИЕ**: Из-за изменений в стандартном сборщике (collator), используемом классом `SFTTrainer` при построении набора данных, токен EOS (который в Phi-3 совпадает с токеном PAD) также маскировался в метках, что приводило к невозможности корректной остановки генерации токенов моделью.

Чтобы устранить это изменение, мы можем назначить токен UNK в качестве токена PAD, чтобы токен EOS стал уникальным и, следовательно, не маскировался как часть меток.

***

In [15]:
from huggingface_hub import snapshot_download

# Загружаем все файлы в локальную директорию
snapshot_download(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    local_dir="./phi3-mini-tokenizer",
    allow_patterns=[
        "tokenizer.json",
        "tokenizer_config.json",
        "special_tokens_map.json",
        "vocab.json",
        "merges.txt",
        "added_tokens.json"
    ]
)

# Затем загружаем из локальной директории
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./phi3-mini-tokenizer")


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [16]:
# tokenizer = AutoTokenizer.from_pretrained(repo_id)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

tokenizer.chat_template


"{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}"

Не обращайте внимания на кажущийся излишне сложным шаблон (я добавил переносы строк и отступы, чтобы его было легче читать). Он просто организует сообщения в связный блок с соответствующими тегами, как показано ниже (`tokenize=False` гарантирует, что мы получим читаемый текст вместо числовой последовательности идентификаторов токенов)

In [17]:
print(tokenizer.apply_chat_template(messages, tokenize=False))


<|user|>
The birch canoe slid on the smooth planks.<|end|>
<|assistant|>
On the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|>
<|endoftext|>


Обратите внимание, что каждое взаимодействие обрамлено либо токенами `<|user|>`, либо `<|assistant|>` в начале и `<|end|>` в конце. Более того, токен `<|endoftext|>` указывает на конец всего блока.

У разных моделей будут разные шаблоны и токены для обозначения начала и конца предложений и блоков.

Теперь мы готовы приступить к собственно тонкой настройке!

## Тонкая настройка с помощью SFTTrainer

**Тонкая настройка модели**, большой или не очень, следует точно **той же процедуре обучения, что и обучение модели с нуля**. Мы могли бы написать собственный цикл обучения на чистом PyTorch или использовать `Trainer` от Hugging Face для тонкой настройки нашей модели.

Однако гораздо проще использовать `SFTTrainer` (который, кстати, использует `Trainer` внутри), поскольку он позаботится о большинстве мелких деталей за нас, при условии, что мы предоставим ему следующие четыре аргумента:

*   модель;
*   токенизатор;
*   набор данных;
*   объект конфигурации.

Первые три элемента у нас уже есть; давайте поработаем над последним.

### SFTConfig

Существует множество параметров, которые мы можем установить в объекте конфигурации. Мы разделили их на четыре группы:

*   Параметры оптимизации **использования памяти**, связанные с **накоплением градиентов и контрольными точками**;
*   Аргументы, связанные с **набором данных**, такие как `max_seq_length`, требуемая вашими данными, и упаковываете ли вы последовательности;
*   Типичные **параметры обучения**, такие как `learning_rate` и `num_train_epochs`;
*   Параметры **окружения и логирования**, такие как `output_dir` (это будет имя модели, если вы решите загрузить ее на Hugging Face Hub после обучения), `logging_dir` и `logging_steps`.

Хотя *скорость обучения* является очень важным параметром (в качестве отправной точки можно попробовать скорость обучения, использовавшуюся при первоначальном обучении базовой модели), именно **максимальная длина последовательности** с большей вероятностью может вызвать **проблемы с нехваткой памяти**.

Всегда выбирайте минимально возможную `max_seq_length`, которая имеет смысл для вашего случая использования. В нашем случае предложения — как на английском, так и в манере речи Йоды — довольно короткие, и последовательности из 64 токенов более чем достаточно, чтобы охватить промпт, завершение и добавленные специальные токены.

<blockquote class="tip">
  <p>
    Flash Attention (который, к сожалению, не поддерживается в Colab) обеспечивает большую гибкость при работе с длинными последовательностями, позволяя избежать потенциальной проблемы ошибок нехватки памяти (OOM).
  </p>
</blockquote>

***

**ВАЖНОЕ ОБНОВЛЕНИЕ**: Выпуск `trl` версии 0.20 принес несколько изменений в `SFTConfig`:
*   упаковка выполняется иначе, чем раньше, если не установлено `packing_strategy='wrapped'`;
*   аргумент `max_seq_length` был переименован в `max_length`;
*   `bf16` по умолчанию имеет значение `True`, но на момент этого обновления (октябрь 2025) не проверялось, доступен ли тип BF16 на самом деле или нет, поэтому он теперь включен в конфигурацию.

***

In [18]:
sft_config = SFTConfig(
    ## ГРУППА 1: Использование памяти
    # Эти аргументы позволят максимально эффективно использовать оперативную память вашего GPU
    # Контрольные точки градиента
    gradient_checkpointing=True,
    # это экономит ОЧЕНЬ много памяти
    # Установите это, чтобы избежать исключений в новых версиях PyTorch
    gradient_checkpointing_kwargs={'use_reentrant': False},
    # Накопление градиента / Размер пакета
    # Фактический пакет (для обновления) такой же (1x), как размер микро-пакета
    gradient_accumulation_steps=1,
    # Начальный размер (микро-)пакета
    per_device_train_batch_size=16,
    # Если размер пакета вызывает OOM, он уменьшается вдвое, пока не заработает
    auto_find_batch_size=True,

    ## ГРУППА 2: Связанные с набором данных
    max_length=64, # переименован в v0.20
    # Набор данных
    # Упаковка набора данных означает, что заполнение не требуется
    packing=True,
    packing_strategy='wrapped', # добавлено для приближения к исходному поведению упаковки

    ## ГРУППА 3: Типичные параметры обучения
    num_train_epochs=10,
    learning_rate=3e-4,
    # Оптимизатор
    # 8-битный оптимизатор Adam - не сильно помогает, если вы используете LoRA!
    optim='paged_adamw_8bit',

    ## ГРУППА 4: Параметры логирования
    logging_steps=10,
    logging_dir='./logs',
    output_dir='./phi3-mini-yoda-adapter',
    report_to='none',

    # гарантирует, что bf16 (новый стандарт) используется только когда он действительно доступен
    bf16=torch.cuda.is_bf16_supported(including_emulation=False)
)


## `SFTTrainer`

In [19]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=dataset,
)


### `SFTTrainer`

<blockquote style="quotes: none !important;">
  <p>
    <em>"Настало время тренировки!"</em>
    <br>
    <br>
    Халк
  </p>
</blockquote>

***

**ВАЖНОЕ ОБНОВЛЕНИЕ**: До версии 0.23 `trl` существовала известная проблема, когда обучение завершалось неудачей, если конфигурация LoRA уже была применена к модели, поскольку тренер замораживал всю модель, включая адаптеры.

Если модель уже содержала адаптеры, как в нашем случае, обучение работало только при использовании базовой исходной модели (`model.base_model.model`) вместе с аргументом `peft_config`.

Эта проблема была исправлена в версии 0.23.1 `trl`, выпущенной в октябре 2025 года.

***

Теперь мы можем, наконец, создать экземпляр тренера для обучения с учителем (supervised fine-tuning trainer):

In [20]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))


In [21]:
batch['input_ids'][0], batch['labels'][0]


(tensor([ 3974, 29892,  4337,   278,   325,   271, 29892,   366,  1818, 29889,
         32007, 32000, 32010,   450,   289,   935,   310,   278,   282,   457,
          5447,   471,   528,  4901,   322,  6501, 29889, 32007, 32001, 26399,
          1758,  4317, 29889,  1383,  4901,   322,  6501, 29892,   278,   289,
           935,   310,   278,   282,   457,  5447,   471, 29889, 32007, 32000,
         32010,   951,  5989,  2507, 17354,   322, 13328,   297,   278,  6416,
         29889, 32007, 32001,   512], device='cuda:0'),
 tensor([ 3974, 29892,  4337,   278,   325,   271, 29892,   366,  1818, 29889,
         32007, 32000, 32010,   450,   289,   935,   310,   278,   282,   457,
          5447,   471,   528,  4901,   322,  6501, 29889, 32007, 32001, 26399,
          1758,  4317, 29889,  1383,  4901,   322,  6501, 29892,   278,   289,
           935,   310,   278,   282,   457,  5447,   471, 29889, 32007, 32000,
         32010,   951,  5989,  2507, 17354,   322, 13328,   297,   278,  64

`SFTTrainer` уже предварительно обработал наш набор данных, поэтому мы можем заглянуть внутрь и посмотреть, как был собран каждый мини-пакет:

In [22]:
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.


Step,Training Loss
10,2.867000
20,1.839400
30,1.612000
40,1.525000
50,1.402200
60,1.297500
70,1.195700
80,1.001400
90,0.903600
100,0.638700


TrainOutput(global_step=220, training_loss=0.8383120309222828, metrics={'train_runtime': 100.7582, 'train_samples_per_second': 33.843, 'train_steps_per_second': 2.183, 'total_flos': 4890970340720640.0, 'train_loss': 0.8383120309222828, 'epoch': 10.0})

## Запрос к модели

Теперь наша модель должна быть способна генерировать предложения в стиле Йоды в ответ на любое короткое предложение, которое мы ей дадим.

Итак, модель требует, чтобы ее входные данные были правильно отформатированы. Нам нужно построить список "сообщений" — в данном случае от `user` — и предложить модели ответить, указав, что сейчас ее очередь писать.

Для этого служит аргумент `add_generation_prompt`: он добавляет `<|assistant|>` в конец диалога, чтобы модель могла предсказать следующее слово — и продолжать делать это, пока не предскажет токен `<|endoftext|>`.

Вспомогательная функция ниже формирует сообщение (в диалоговом формате) и **применяет шаблон чата** к нему, **добавляя промпт генерации** в его конец.

In [23]:
def gen_prompt(tokenizer, sentence):
    converted_sample = [
        {"role": "user", "content": sentence},
    ]
    prompt = tokenizer.apply_chat_template(converted_sample,
                                           tokenize=False,
                                           add_generation_prompt=True)
    return prompt


Давайте попробуем сгенерировать промпт для примера предложения:

In [24]:
sentence = 'The Force is strong in you!'
prompt = gen_prompt(tokenizer, sentence)
print(prompt)


<|user|>
The Force is strong in you!<|end|>
<|assistant|>



Промпт выглядит правильным; давайте используем его для генерации завершения. Вспомогательная функция ниже выполняет следующие действия:

*   **Токенизирует промпт** в тензор идентификаторов токенов (`add_special_tokens` установлен в `False`, потому что токены уже были добавлены шаблоном чата).
*   Устанавливает модель в **режим оценки**.
*   Вызывает метод `generate()` модели для **создания вывода** (сгенерированных идентификаторов токенов).
    *   Если модель обучалась с использованием смешанной точности, мы оборачиваем генерацию в менеджер контекста `autocast()`, который автоматически обрабатывает преобразование между типами данных.
*   **Декодирует сгенерированные идентификаторы токенов** обратно в читаемый текст.

In [25]:
def generate(model, tokenizer, prompt, max_new_tokens=64, skip_special_tokens=False):
    tokenized_input = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").to(model.device)

    model.eval()
    # если обучение проводилось со смешанной точностью, используем контекст autocast
    ctx = torch.autocast(device_type=model.device.type, dtype=model.dtype) \
          if model.dtype in [torch.float16, torch.bfloat16] else nullcontext()
    with ctx:  
        generation_output = model.generate(**tokenized_input,
                                           eos_token_id=tokenizer.eos_token_id,
                                           max_new_tokens=max_new_tokens)

    output = tokenizer.batch_decode(generation_output,
                                    skip_special_tokens=skip_special_tokens)
    return output[0]


Теперь мы можем, наконец, опробовать нашу модель и посмотреть, действительно ли она способна генерировать речь в стиле Йоды.

In [26]:
print(generate(model, tokenizer, prompt))


<|user|> The Force is strong in you!<|end|><|assistant|> Strong in you, the Force is! Yes, hrrrm.<|end|><|endoftext|>


Потрясающе! Это работает! Модель говорит как Йода. Хрррммм.

Поздравляю, вы выполнили тонкую настройку своей первой LLM!

Теперь у вас есть небольшой адаптер, который можно загрузить в экземпляр модели Phi-3 Mini 4K Instruct, чтобы превратить ее в переводчика на язык Йоды! Как это круто?

### Сохранение адаптера

После завершения обучения вы можете сохранить адаптер (и токенизатор) на диск, вызвав метод `save_model()` тренера. Он сохранит все в указанную папку:

In [27]:
trainer.save_model('local-phi3-mini-yoda-adapter')


Сохраненные файлы включают:

*   конфигурацию адаптера (`adapter_config.json`) и веса (`adapter_model.safetensors`) — сам адаптер занимает всего 50 МБ;
*   аргументы обучения (`training_args.bin`);
*   токенизатор (`tokenizer.json` и `tokenizer.model`), его конфигурацию (`tokenizer_config.json`) и специальные токены (`added_tokens.json` и `speciak_tokens_map.json`);
*   файл README.

In [28]:
os.listdir('local-phi3-mini-yoda-adapter')


['tokenizer_config.json',
 'adapter_model.safetensors',
 'special_tokens_map.json',
 'adapter_config.json',
 'training_args.bin',
 'chat_template.jinja',
 'README.md',
 'tokenizer.json']

Если вы хотите поделиться своим адаптером со всеми, вы также можете загрузить его на Hugging Face Hub. Сначала войдите в систему, используя токен с разрешением на запись:

In [29]:
# from huggingface_hub import login
# login()


Приведенный выше код попросит вас ввести токен доступа:

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch0/hub0.png?raw=True)
<center>Рисунок 0.1 - Вход в Hugging Face Hub</center>

Успешный вход должен выглядеть так (обратите внимание на разрешения):

![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch0/hub1.png?raw=True)
<center>Рисунок 0.2 - Успешный вход</center>

Затем вы можете использовать метод `push_to_hub()` тренера, чтобы загрузить все в свою учетную запись в Hub. Модель будет названа в соответствии с аргументом `output_dir` аргументов обучения:

In [30]:
# trainer.push_to_hub()


Готово! Наша модель теперь доступна всем, и любой может использовать ее для перевода английского языка на манер речи Йоды.

На этом все!